# Tracing annotations library

In [ ]:
package 'TracingAnnotations' {
	 
	import Base::*;
	import ScalarValues::*;
	import IntegerFunctions::*;
	
	attribute def ConfidenceTracing {
		attribute confidence : Real;
 		assert constraint confBtz0n1  { confidence >= 0.0 && confidence <= 1.0 } 
        attribute cost : Real;
        attribute agent : Agent;
		attribute impact : Anything[*];
        attribute description : String;
	}
	
	attribute def Agent {
		attribute description : String;
	}
	
	attribute def HumanAgent :> Agent {
		attribute role : String;
	}
	
	attribute def MachineAgent :> Agent {
		attribute type : String;
	}
    
    attribute def TraceType {
        attribute tracetype : String;
    }

    enum def TraceTypes :> String {
        typeA = "ApplicationType"; 
        typeE = "EngineeringType"; 
    }
 }

# Example (Architecture / Library)

In [39]:
package eDrone_Library {
        
    attribute def ElectricEnergy;
    attribute def BatteryCapacity :> ScalarValues::Integer;
    attribute def Speed :> ScalarValues::Integer;
    
    port def PowerOutPort {
        out energy : ElectricEnergy;
    }
    
    interface def PowerInterface {
        end supplierPort : PowerOutPort;
        end consumerPort : ~PowerOutPort;
    }
}

package eDrone_Definitions {

    import eDrone_Library::*;   
    
    part def Wing {
         size : ScalarValues::Integer;
    }
    
    part def Battery {
         capacity : BatteryCapacity;
    }
    
    part def Engine;
}

package eDrone_LogicalArchitecture_ExplosionExample {   
    import eDrone_Definitions::*;
    import eDrone_Library::*;
        
    part eDrone {    
        part engine : Engine;
        part lugBolt42;
        part lugBolt23;

        part frontAxis {
            part frontWing[2] {
                /*-----------------------------
                 * Definition of the variation
                 * ----------------------------*/
                variation part lugBolt[6] {
                    variant part lugBolt23;
                    variant part lugBolt42;  
                }
                part boltCircle;
                connect boltCircle to lugBolt;
            }
            part housing;            
            connect housing to frontWing;
        }
        
        part rearAxis {
            part rearWing[2] {
                part lugBolt[6];
                part boltCircle;
                connect boltCircle to lugBolt;
            }
            part housing;    
            connect housing to rearWing;
        }        
        connect engine to frontAxis;
    }
}

Package eDrone_Library (86a9900f-2cd2-49e9-816e-3c0f3d01c6b3)
Package eDrone_Definitions (0c2c6706-4b9f-4e4a-b006-b7f76870b565)
Package eDrone_LogicalArchitecture_ExplosionExample (21cb2c54-faca-4ac5-9535-869b3245c16c)


# Requirements

In [2]:
package eDroneRequirement_Definitions {
    
    import eDrone_Definitions::*;
    
    requirement def BatteryCapacityReqDef {
        doc /* The actual battery capacity shall be greater than or equal
             * to the required capacity. */

            attribute capacityActual : BatteryCapacity;
            attribute capacityRequired : BatteryCapacity;
        
            require constraint{ capacityActual <= capacityRequired }
    }
    
    requirement def MaxSpeedReqDef {
        doc /* The maximum speed of the vehicle shall be 
             * not greater than the required maximum speed. */
            
            attribute maxSpeedVehicle : Speed;
            attribute maxSpeedRequired : Speed;
        
            require constraint{ maxSpeedVehicle <= maxSpeedRequired }
    }
    
}

package eDrone_Requirements {
    
    import eDroneRequirement_Definitions::*;
    import eDrone_LogicalArchitecture::*;
    
    requirement eDroneSpecification {
        doc /* Requirement Specification of the eDrone  */
        subject eDrone :>> eDrone;
        require eDroneBatteryCapacity;
        require eDroneMaxSpeed;
    }
    
    requirement id 'REQ.B.1' eDroneBatteryCapacity : BatteryCapacityReqDef {
        subject eDrone :>> eDrone;
        attribute :>> capacityRequired = 50;
        attribute :>> capacityActual = eDrone.battery.capacity;
    }
    
    requirement id 'REQ.V.1' eDroneMaxSpeed : MaxSpeedReqDef {
        subject eDrone :>> eDrone;
        attribute :>> maxSpeedRequired = 140;
        attribute :>> maxSpeedVehicle = eDrone.maxSpeed;
    }
    
}

package tracing_MetasConnections {
    import eDrone_Requirements::*;
    import eDrone_LogicalArchitecture::*;
    /* Here we add a simple connection from Req2Feature for the Tracing use case*/
    
    connection rq1ToWing connect eDroneMaxSpeed to eDrone.engine;
}

package eDrone_Library {
        
    attribute def ElectricEnergy;
    attribute def BatteryCapacity :> ScalarValues::Integer;
    attribute def Speed :> ScalarValues::Integer;
    
    port def PowerOutPort {
        out energy : ElectricEnergy;
    }
    
    interface def PowerInterface {
        end supplierPort : PowerOutPort;
        end consumerPort : ~PowerOutPort;
    }
}

package eDrone_Definitions {

    import eDrone_Library::*;   
    
    part def Wing {
         size : ScalarValues::Integer;
    }
    
    part def Battery {
         capacity : BatteryCapacity;
    }
    
    part def Engine;
}


package eDrone_LogicalArchitecture {
       
    import eDrone_Definitions::*;
    import eDrone_Library::*;
    
    part eDrone {    
        maxSpeed :Speed = 142;
        part body;
        part battery {
            attribute capacity : BatteryCapacity;
            port powerOut : PowerOutPort;
        }
        part engine {
            port powerIn : ~PowerOutPort;
        }
        part frontAxis;
        part rearAxis;
        part frontWing : Wing[2];
        part rearWing : Wing[2];
                
        interface : PowerInterface connect 
            supplierPort :> battery.powerOut to 
            consumerPort :> engine.powerIn;
        /*connection eng2front connect engine to frontAxis;
        connection frAx2frWi connect frontAxis to frontWing;
        connection reAx2reWi connect rearAxis to rearWing;
        connection bod2bat   connect body to battery;
        connection bod2eng   connect body to engine;
        connection bod2frAx  connect body to frontAxis;
        connection bod2reAx  connect body to rearAxis;
        
        connection eng2bat_Typed: EngineToBattery connect supplierPort :> battery.powerOut to 
            consumerPort :> engine.powerIn;*/
    }
    
    /*connection def EngineToBattery{
        end: PowerOutPort;
        end: ~PowerOutPort;
    }*/
}


Package eDroneRequirement_Definitions (ae1e26f3-9cf9-4ccf-a1df-3d1b42472dbe)
Package eDrone_Requirements (4bacd473-9c56-444f-9404-36c7d22f27a2)
Package tracing_MetasConnections (e0b3f5f1-7ab5-465d-ba97-bbfe5091582d)
Package eDrone_Library (6f9bfad2-91ef-45af-ae91-84b96155d31a)
Package eDrone_Definitions (1bd833ed-f531-42d7-ad3e-883fe961c911)
Package eDrone_LogicalArchitecture (ac1c7d99-2f9a-46a4-b278-6ab6177d8f8d)


In [59]:
%export eDrone_LogicalArchitecture 

..

Download

In [1]:
%export tracing_MetasConnections

ERROR:Couldn't resolve reference to Element 'tracing_MetasConnections'


In [ ]:
package eDrone_Example {
	import TracingAnnotations::*;	
    
    import eDroneRequirement_Definitions::*;
    import eDrone_Requirements::*;
    import eDrone_LogicalArchitecture::*;
    import tracing_MetasConnections::*;
    import eDrone_Definitions::*;
    import eDrone_Library::*;

    connection eng2front connect eDrone.engine to eDrone.frontAxis;
    /* Trace link annotations */
       metadata m7159: ConfidenceTracing about eng2front { confidence = 0.55;}
       metadata m1014: TraceType about eng2front { tracetype = "typeC";}
    
    connection frAx2frWi connect eDrone.frontAxis to eDrone.frontWing;
    /* Trace link annotations */
        metadata m71569: ConfidenceTracing about frAx2frWi { confidence = 0.65;}
        metadata m10164: TraceType about frAx2frWi { tracetype = "typeE";}
    
    connection reAx2reWi connect eDrone.rearAxis to eDrone.rearWing;
    connection bod2bat   connect eDrone.body to eDrone.battery;
    connection bod2eng   connect eDrone.body to eDrone.engine;
    connection bod2frAx  connect eDrone.body to eDrone.frontAxis;
    connection bod2reAx  connect eDrone.body to eDrone.rearAxis;

    connection eng2bat_Typed: EngineToBattery connect supplierPort :> eDrone.battery.powerOut to 
        consumerPort :> eDrone.engine.powerIn;
        /* Trace link annotations */
        metadata m7159444: ConfidenceTracing about eng2front { confidence = 0.75;}
        metadata m10814: TraceType about eng2bat_Typed { tracetype = "typeE";}
        metadata m108814: TraceType about eng2bat_Typed { tracetype = "typeC";}
    
    connection def EngineToBattery{
        end: PowerOutPort;
        end: ~PowerOutPort;
    }    
}

In [22]:
%export eDrone_Example

......

Download